## Hands-On Labs 2
### Objectives
- Learn how to align a model's behavior using labelled preference data.


## 下載對應版本的套件 (Install the corresponding version of the packages.)

In [1]:
!pip install bitsandbytes==0.45.0 datasets==3.1.0 peft==0.13.2 trl==0.12.1 accelerate==1.1.1 transformers==4.46.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transfor

## 匯入套件 (Import packages)

In [2]:
import torch
import json
from datasets import Dataset
import pandas as pd
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig
from tqdm.auto import tqdm
from trl import DPOTrainer, DPOConfig

## 準備資料集 (Prepare training data)

In [3]:
# 下載資料 (Download data)
!git clone https://github.com/Baiiiiiiiiii/GenAI_hw6_dataset.git

Cloning into 'GenAI_hw6_dataset'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (4/4), 4.06 KiB | 4.06 MiB/s, done.


In [4]:
# 讀取資料 (Read data)
with open("/content/GenAI_hw6_dataset/labelled_data.json", 'r') as jsonfile:
    full_data = json.load(jsonfile)

with open("/content/GenAI_hw6_dataset/test_prompt.json", 'r') as jsonfile:
    test_data = json.load(jsonfile)

In [5]:
# 準備分詞器 (Prepare tokenizer)
tokenizer = AutoTokenizer.from_pretrained('MediaTek-Research/Breeze-7B-Instruct-v0_1')
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/911k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.79M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [6]:
def data_formulate(data):
    messages = [
        {"role": "system", "content": '回覆請少於20字'},
        {"role": "user", "content": data['prompt']},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

In [7]:
# 設定資料大小 (Set data size)
data_size = 50
training_data = full_data[:data_size]

# 準備訓練資料集 (Prepare the data for the training dataset)
prompt_list = [data_formulate(data) for data in training_data]
chosen_list = [data['support'] for data in training_data[:data_size]]
rejected_list = [data['oppose'] for data in training_data[:data_size]]
train_dataset = Dataset.from_dict({'prompt': prompt_list, 'chosen': chosen_list, 'rejected': rejected_list})

In [8]:
# 查看資料集 (Visualize data)
pd.DataFrame(train_dataset).rename(columns={"chosen": "preferred", "rejected": "non-preferred"})

,prompt,preferred,non-preferred
0,<s>回覆請少於20字 [INST] 日本動漫真人化是否有損原作形象？ [/INST],真人化能夠呈現更真實的角色形象，提升原作魅力。,真人化可能無法完美呈現動畫中的獨特風格，損害原作形象。
1,<s>回覆請少於20字 [INST] 真人化是否能夠擴大動漫在全球的影響力？ [/INST],真人化能夠讓更多非動漫迷接觸作品，擴大影響力。,真人化可能失去動漫的獨特風格，限制影響力擴大。
2,<s>回覆請少於20字 [INST] 真人化是否能夠吸引新觀眾？ [/INST],真人化能夠吸引不熟悉動漫的觀眾，擴大受眾。,真人化可能讓原本的動漫迷感到失望，無法吸引新觀眾。
3,<s>回覆請少於20字 [INST] 真人化是否能夠保留原作故事情節的精髓？ [/INST],真人化有機會更深入挖掘原作故事，保留精髓。,真人化可能因為改編而失去原作故事的深度與精髓。
4,<s>回覆請少於20字 [INST] 真人化是否能夠提升動漫產業的商業價值？ [/INST],真人化能夠開拓更多商業機會，提升產業價值。,真人化可能讓觀眾對原作失去興趣，影響產業價值。
5,<s>回覆請少於20字 [INST] 真人化是否能夠保持原作的文化特色？ [/INST],真人化可以透過場景、服裝等元素保留文化特色。,真人化可能因為文化差異而失去原作獨有的文化魅力。
6,<s>回覆請少於20字 [INST] 真人化是否能夠挑戰技術上的新突破？ [/INST],真人化促使技術創新，挑戰視覺效果上的新高度。,真人化可能因為技術限制而無法達到動畫中的視覺效果。
7,<s>回覆請少於20字 [INST] 真人化是否會受到演員選擇的爭議？ [/INST],演員選擇可因應市場需求，不必受限於動畫形象。,演員選擇可能引起爭議，觀眾難以接受角色塑造。
8,<s>回覆請少於20字 [INST] 真人化是否能夠提高動漫的社會認同度？ [/INST],真人化有機會讓更多人接受動漫，提高社會認同度。,真人化可能因為劇情改編而無法贏得社會認同。
9,<s>回覆請少於20字 [INST] 真人化是否能夠保留原作角色的個性特色？ [/INST],真人化可以透過演員表現保留角色的個性特色。,真人化可能因演員演技或導演選擇而失去角色的原有特色。


## 準備模型 (Load model)

In [9]:
# 下載模型 (Download model)
model = AutoModelForCausalLM.from_pretrained(
    'MediaTek-Research/Breeze-7B-Instruct-v0_1',
    device_map='auto',
    trust_remote_code=True,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    )
)

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

## 查看訓練前模型的輸出 (Check the output of the model before training)

In [10]:
original_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f'Question {id}:\n'+data['prompt'])
    inputs = tokenizer(data_formulate(data), return_tensors="pt").to('cuda')
    generation_config=GenerationConfig(
            do_sample=False,
            max_new_tokens = 200,
            pad_token_id = tokenizer.pad_token_id
    )
    output = model.generate(**inputs, generation_config=generation_config)
    output = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]
    original_model_response.append(output)
    print('Response from original model:\n'+output+'\n')

  0%|          | 0/10 [00:00<?, ?it/s]

Question 1:
真人化是否能改善日本漫畫的全球可及性？
Response from original model:
真人化可能會提高日本漫畫的全球可及性，因真人版電影或劇集可以吸引更多非漫畫讀者的注意，並提供不同的體驗。然而，這取決於真人化作品的品質、行銷策略和市場接受度。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？
Response from original model:
真人化可能會影響年輕一代對日本漫畫的看法，使他們更容易接受和理解故事和角色，並吸引更多人關注和支持日本漫畫文化。然而，個人喜好和文化差異可能導致不同的影響。

Question 3:
真人化是否能提升原作漫畫的文學價值？
Response from original model:
真人化可能會提升原作漫畫的知名度和影響力，但文學價值本身可能因個人喜好和文化差異而異。真人化可能帶來更多觀眾，但文學價值取決於原作的故事、人物和主題，而非真人化形式。

Question 4:
真人化是否有助於保護和保存日本漫畫的傳統？
Response from original model:
真人化可能有助於提高日本漫畫的知名度和吸引更多觀眾，但是否真正保護和保存傳統尚需視真人化作品是否尊重原作精神和文化價值。

Question 5:
真人化是否有助於提升日本漫畫行業的經濟效益？
Response from original model:
真人化可能有助於提升日本漫畫行業的經濟效益，因真人版電影或劇集可以吸引更多觀眾，增加收入來源。然而，成功與否取決於作品的品質、行銷策略和市場接受度。

Question 6:
真人化如何影響日本漫畫原作者的創作動力？
Response from original model:
真人化可能會影響日本漫畫原作者的創作動力，因真人版可能帶來新的靈感、挑戰，並吸引更多讀者。然而，個人感受和反應各異，有些作者可能因真人化而更投入創作，而其他人可能因個人喜好或對真人化看法不同而影響其動力。

Question 7:
真人化是否對漫畫原作的忠實粉絲公平？
Response from original model:
真人化可能會影響忠實的漫畫原作粉絲，因真人版可能有不同的故事改編、角色設定或表現方式，但個人喜好不同，仍有可能欣賞。

Question 8:
真

## 設定訓練參數 (Set training parameters)

In [11]:
training_args = DPOConfig(
    output_dir='./',
    per_device_train_batch_size=1,
    num_train_epochs=2,
    gradient_accumulation_steps=8,
    gradient_checkpointing=False,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps = 1,
    warmup_ratio = 0.1,
    report_to = 'none'
)

In [12]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [13]:
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    beta=0.1,
    train_dataset=train_dataset,
    processing_class=tokenizer,
    peft_config=peft_config,
)

Extracting prompt from train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

## 開始訓練 (Start training)

In [14]:
dpo_trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.693100
2,0.693100
3,0.619800
4,0.401600
5,0.211400
6,0.185800
7,0.048200
8,0.016200
9,0.008300
10,0.014300


TrainOutput(global_step=12, training_loss=0.24222829677940658, metrics={'train_runtime': 320.9035, 'train_samples_per_second': 0.312, 'train_steps_per_second': 0.037, 'total_flos': 0.0, 'train_loss': 0.24222829677940658, 'epoch': 1.92})

## 查看訓練後模型的輸出 (Check the output of the model after training)

In [15]:
trained_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f'Question {id}:\n'+data['prompt'])
    inputs = tokenizer(data_formulate(data), return_tensors="pt").to('cuda')
    generation_config=GenerationConfig(
            do_sample=False,
            max_new_tokens = 200,
            pad_token_id = tokenizer.pad_token_id
    )
    output = model.generate(**inputs, generation_config=generation_config)
    output = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]
    trained_model_response.append(output)
    print('Response from trained model:\n'+output+'\n')

  0%|          | 0/10 [00:00<?, ?it/s]

Question 1:
真人化是否能改善日本漫畫的全球可及性？
Response from trained model:
真人化可以提高日本漫畫的全球可及性，吸引更多觀眾並拓展市場。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？
Response from trained model:
真人化能提升年輕一代對日本漫畫的興趣，加深印象和理解。

Question 3:
真人化是否能提升原作漫畫的文學價值？
Response from trained model:
真人化能拓展更多表現形式，提升文學價值。

Question 4:
真人化是否有助於保護和保存日本漫畫的傳統？
Response from trained model:
真人化有助於將日本漫畫的傳統保護和保存，同時促進文化交流和藝術發展。

Question 5:
真人化是否有助於提升日本漫畫行業的經濟效益？
Response from trained model:
真人化可以提升日本漫畫行業的經濟效益，吸引更多觀眾並拓展不同市場。

Question 6:
真人化如何影響日本漫畫原作者的創作動力？
Response from trained model:
真人化可以增強日本漫畫原作者的創作動力，提供新的挑戰和機會，同時吸引更多讀者，拓展故事世界。

Question 7:
真人化是否對漫畫原作的忠實粉絲公平？
Response from trained model:
真人化可以為忠實的漫畫原作的粉絲帶來更廣泛的認識和享受，但個人喜好和感受仍有所差異。整體而言，真人化提供了更多元的體驗機會。

Question 8:
真人化是否能夠促進日本漫畫的創新和多樣性？
Response from trained model:
真人化可以促進日本漫畫的創新和多樣性，帶來更多元的故事和角色設計。

Question 9:
真人化是否有助於擴大動漫文化的市場份額？
Response from trained model:
真人化有助於吸引更多元的觀眾群，擴大動漫文化的市場份額。

Question 10:
真人化是否有助於提高日本漫畫在全球的競爭力？
Response from trained model:
真人化可以提高日本漫畫在全球的競爭力，擴大受眾群並創造更多商機。

